# Código Python para buscar Comentarios Youtube e Gerar os arquivos .csv no formato normalizado e com label 

In [ ]:
import os
import pandas as pd
import tkinter as tk
from tkinter import ttk
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

def search_youtube_comments(api_key, video_id, max_results=100):
    youtube = build('youtube', 'v3', developerKey=api_key)
    comments = []
    next_page_token = None

    while True:
        try:
            response = youtube.commentThreads().list(
                part='snippet,replies',
                videoId=video_id,
                maxResults=max_results,
                pageToken=next_page_token
            ).execute()

            for item in response.get('items', []):
                comment = item['snippet']['topLevelComment']['snippet']
                comments.append({
                    'comment_id': item['id'],
                    'author': comment['authorDisplayName'],
                    'url': f"https://www.youtube.com/watch?v={video_id}&lc={item['id']}",
                    'date': comment['publishedAt'],
                    'likes': comment.get('likeCount', 0),
                    'shares': 0,  # No direct field for shares in YouTube API for comments
                    'comments': comment.get('totalReplyCount', 0),
                    'content': comment['textDisplay'],
                    'service': 'YouTube',
                    'sentiment': 'NEUTRAL',  # Placeholder sentiment, update based on your sentiment analysis
                    'engagement': 0,  # Placeholder engagement, update as needed
                    **comment  # Include all additional comment fields
                })

                # Process replies
                for reply in item.get('replies', {}).get('comments', []):
                    reply_snippet = reply['snippet']
                    comments.append({
                        'comment_id': reply['id'],
                        'author': reply_snippet['authorDisplayName'],
                        'url': f"https://www.youtube.com/watch?v={video_id}&lc={reply['id']}",
                        'date': reply_snippet['publishedAt'],
                        'likes': reply_snippet.get('likeCount', 0),
                        'shares': 0,  # No direct field for shares in YouTube API for comments
                        'comments': 0,  # Replies to replies not handled in this example
                        'content': reply_snippet['textDisplay'],
                        'service': 'YouTube',
                        'sentiment': 'NEUTRAL',  # Placeholder sentiment, update based on your sentiment analysis
                        'engagement': 0,  # Placeholder engagement, update as needed
                        **reply_snippet  # Include all additional reply fields
                    })

            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break

        except HttpError as e:
            print(f"Erro ao buscar comentários: {e}")
            raise

    return comments

def convert_sentiment(sentiment):
    sentiment_mapping = {
        'NEUTRAL': 0,
        'POSITIVE': 1,
        'NEGATIVE': 2
    }
    return sentiment_mapping.get(sentiment, 0)

def save_comments_to_csv(comments, output_directory):
    for index, comment in enumerate(comments):
        video_id = comment['comment_id']
        service = comment['service']
        date = pd.to_datetime(comment['date']).strftime('%Y%m%d%H%M')
        sentiment = convert_sentiment(comment['sentiment'])
        index_formatted = str(index).zfill(7)

        filename = f"{service}_{video_id}_{date}_{sentiment}_{index_formatted}.csv"
        filepath = os.path.join(output_directory, filename)

        df_single_row = pd.DataFrame([comment])
        df_single_row.to_csv(filepath, index=False)

def run_code(api_key, video_id, output_directory):
    comments = search_youtube_comments(api_key, video_id)
    save_comments_to_csv(comments, output_directory)

def on_submit():
    api_key = api_key_entry.get()
    video_id = video_id_entry.get()
    output_directory = output_directory_entry.get()

    run_code(api_key, video_id, output_directory)

root = tk.Tk()
root.title("YouTube Comments Downloader")

frame = ttk.Frame(root, padding="10")
frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

api_key_label = ttk.Label(frame, text="API Key:")
api_key_label.grid(row=0, column=0, sticky=tk.W)
api_key_entry = ttk.Entry(frame, width=30)
api_key_entry.grid(row=0, column=1, sticky=(tk.W, tk.E))

video_id_label = ttk.Label(frame, text="Video ID:")
video_id_label.grid(row=1, column=0, sticky=tk.W)
video_id_entry = ttk.Entry(frame, width=30)
video_id_entry.grid(row=1, column=1, sticky=(tk.W, tk.E))

output_directory_label = ttk.Label(frame, text="Output Directory:")
output_directory_label.grid(row=2, column=0, sticky=tk.W)
output_directory_entry = ttk.Entry(frame, width=30)
output_directory_entry.grid(row=2, column=1, sticky=(tk.W, tk.E))

submit_button = ttk.Button(frame, text="Submit", command=on_submit)
submit_button.grid(row=3, column=0, columnspan=2)

root.mainloop()


# Update de codigo para ajustar entendimento da ultima reunião de 20240613. #